<a href="https://colab.research.google.com/github/farhanhubble/discover-drl/blob/master/Rediscovering_RL_Notebook_0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
agent = '⚽'
opponent = '👕'
goal = '🥅'

arena = [['⚽', ' ' , '👕', ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , ' ' ],
         [' ' , ' ' , ' ' , '👕'],
         [' ' , '👕', ' ' , '🥅']]

In [117]:
import numpy as np

class Foolsball(object):

  def __to_state__(self,row,col):
    """Convert from integer state to indices (row,col)."""
    return row*self.n_cols + col

  def __to_indices__(self, state):
    """Convert indices(row,col) to state (single integer)."""
    row = state // self.n_cols
    col = state % self.n_cols
    return row,col

  def __deserialize__(self,map:list,agent:str,opponent:str, goal:str):
    """Convrt a string representation of a map into a 2D numpy array
    Param map: list of lists of strings representing the player, opponents and goal.
    Param agent: string representing the agent on the map 
    Param opponent: string representing every instance of an opponent player
    Param goal: string representing the location of the goal on the map
    """
    ## Capture dimensions and map.
    self.n_rows = len(map)
    self.n_cols = len(map[0])
    self.n_states = self.n_rows * self.n_cols 
    self.map = np.asarray(map)

    ## Store string representations for printing the map, etc.
    self.agent_repr = agent
    self.opponent_repr  = opponent
    self.goal_repr = goal

    ## Find initial state, the desired goal state and the state of the opponents. 
    self.init_state = None
    self.goal_state = None
    self.opponents_states = []

    for row in range(self.n_rows):
      for col in range(self.n_cols):
        if map[row][col] == agent:
          # Store the initial state outside the map.
          # This helps in quickly resetting the game to the initial state and
          # also simplifies printing the map independent of the agent's state. 
          self.init_state = self.__to_state__(row,col)
          self.map[row,col] = ' ' 
        
        elif map[row][col] == opponent:
          self.opponents_states.append(self.__to_state__(row,col))

        elif map[row][col] == goal:
          self.goal_state = self.__to_state__(row,col)

    assert self.init_state is not None, print(f"Map {map} does not specify an agent {agent} location")
    assert self.goal_state is not None,  print(f"Map {map} does not specify a goal {goal} location")
    assert self.opponents_states,  print(f"Map {map} does not specify any opponents {opponent} location")

    return self.init_state


  def __get_next_state_on_action__(self,state,action):
    """Return next state based on current state and action."""
    row, col = self.__to_indices__(state)
    action_to_index_delta = {'n':[-1,0], 'e':[0,+1], 'w':[0,-1], 's':[+1,0]}

    row_delta, col_delta = action_to_index_delta[action]
    new_row , new_col = row+row_delta, col+col_delta

    ## Return current state if next state is invalid
    ## The caller need to check for this
    if not(0<=new_row<self.n_rows) or not(0<=new_col<self.n_cols):
      return state  
    
    return self.__to_state__(new_row, new_col)


  def __get_reward_for_transition__(self,state,next_state):
    """ Return the reward based on the transition from current state to next state. """
    ## Transition rejected due to illegal action (move)
    if next_state == self.state:
      reward = self.rewards['outside']
    
    ## Goal!
    elif next_state == self.goal_state:
      reward = self.rewards['goal']
    
    ## Ran into opponent. 
    elif next_state in self.opponents_states:
      reward = self.rewards['opponent']

    ## Made a safe and valid move.   
    else:
      reward = self.rewards['unmarked']

    return reward


  def __is_terminal_state__(self, state):
    return (state == self.goal_state) or (state in self.opponents_states) 

  
  def __init__(self,map,agent,opponent,goal):
    """Spawn the world, create variables to track state and actions."""
    # We just need to track the location of the agent (the ball)
    # Everything else is static and so a potential algorithm doesn't 
    # have to look at it. The `done` flag terminal states.
    self.state = self.__deserialize__(map,agent,opponent,goal)
    self.done = False
    self.actions = ['n','e','w','s']

    # Build a trnasition table (dict of dicts), mapping every (state,action)
    # pair to the next state. This defines the one-step dynamics of the 
    # environment 
    self.transitions = self.__install_transition_table__()


    # Set up the rewards
    self.default_rewards = {'unmarked':-1, 'opponent':-5, 'outside':-1, 'goal':+5}
    self.set_rewards(self.default_rewards)

  
  def __install_transition_table__(self):
    ## Create a dictionary of dictionaries that can map every (state,action) pair
    ## to the corresponding next state.
    transitions = {s:{a:None for a in self.actions} for s in range(self.n_states)}
    for s in range(self.n_states):
      for a in self.actions:
        transitions[s][a] = self.__get_next_state_on_action__(s,a)


  def reset(self):
    """Reset the environment to its initial state."""
    # There's really just two things we need to reset: the state, which should
    # be reset to the initial state, and the `done` flag which should be 
    # cleared to signal that we are not in a terminal state anymore, even if we 
    # were earlier. 
    self.state = self.init_state
    self.done  = False
    return self.state

  
  def set_rewards(self,rewards):
    if not self.state == self.init_state:
      print('Warning: Setting reward while not in initial state! You may want to call reset() first.')
    for key in self.default_rewards:
      assert key in rewards, print(f'Key {key} missing from reward.') 
    self.rewards = rewards

  
  def step(self,action):
    """Simulate state transition based on current state and action received."""
    assert not self.done, \
    print(f'You cannot call step() in a terminal state({self.state}). Check the "done" flag before calling step() to avoid this.')
    next_state = self.__get_next_state_on_action__(self.state, action)

    reward = self.__get_reward_for_transition__(self.state, next_state)
    done = self.__is_terminal_state__(next_state)

    self.state, self.done = next_state, done
    
    return next_state, reward, done


  def render(self):
    """Pretty-print the environment and agent."""
    ## Create a copy of the map and change data type to accomodate
    ## 3-character strings
    _map = np.array(self.map, dtype='<U3')

    ## Mark unoccupied positions with special symbol.
    ## And add extra spacing to align all columns.
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        if _map[row,col] == ' ':
          _map[row,col] = ' + '
        
        elif _map[row,col] == self.opponent_repr: 
          _map[row,col] =  self.opponent_repr + ' '
        
        elif _map[row,col] == self.goal_repr:
          _map[row,col] = ' ' + self.goal_repr + ' '
      
    ## If current state overlaps with the goal state or one of the opponents'
    ## states, susbstitute a distinct marker.
    if self.state == self.goal_state:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' 🏁 '
    elif self.state in self.opponents_states:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ❗ '
    else:
      r,c = self.__to_indices__(self.state)
      _map[r,c] = ' ' + self.agent_repr
    
    for row in range(_map.shape[0]):
      for col in range(_map.shape[1]):
        print(f' {_map[row,col]} ',end="")
      print('\n') 
    
    print()



In [118]:
foolsball = Foolsball(arena, agent, opponent, goal)

In [119]:
foolsball.render()

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  




In [120]:
## Move: n,s,e,w
## Reset: r
## Exit: x
while True:
  try:
    act = input('>>')

    if act in foolsball.actions:
      print(foolsball.step(act))
      print()
      foolsball.render()
    elif act == 'r':
      print(foolsball.reset())
      print()
      foolsball.render()
    elif act == 'x':
      break
    else:
      print(f'Invalid input:{act}')
  except Exception as e:
    print(e)

>>x


In [122]:
## Reward and return
path1 = ['e','s','e','s','s','s','e']
path2 = ['s','e','e','s','s','s','e']
path3 = ['s','s','s','e','e','s','e']
path4 = ['s','s','s','s','n','e','e','s','e']

In [123]:
def get_return(path):
  foolsball.reset()
  foolsball.render()
  _return_ = 0
  for act in path: 
    next_state, reward, done = foolsball.step(act)
    foolsball.render()
    _return_ += reward
    
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [124]:
get_return(path1)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    ⚽  👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): -1


In [125]:
get_return(path2)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): -1


In [126]:
get_return(path3)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): -1


In [127]:
get_return(path4)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  ⚽  👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    + 

In [128]:
## Different reward structure
foolsball.set_rewards({'unmarked':0, 'opponent':-5, 'outside':-1, 'goal':+5})

In [129]:
get_return(path1)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    ⚽  👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): 5


In [130]:
get_return(path4)

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  ⚽  👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    + 

In [131]:
def get_discounted_return(path, gamma=0):
  foolsball.reset()
  foolsball.render()
  _return_ = 0
  discount_coeff = 1
  for act in path: 
    next_state, reward, done = foolsball.step(act)
    foolsball.render()
    _return_ += discount_coeff*reward
    discount_coeff *= gamma    
    if done:
      break
    
  print(f'Return (accumulated reward): {_return_}')

In [132]:
HYPER_PARAMS = {'gamma':0.9}

In [133]:
get_discounted_return(path1, HYPER_PARAMS['gamma'])

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    ⚽  👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    ⚽   +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    ⚽  👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    ⚽   +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🏁  


Return (accumulated reward): 2.6572050000000007


In [134]:
get_discounted_return(path4, HYPER_PARAMS['gamma'])

  ⚽   +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  ⚽   +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  ⚽  👕    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  ⚽  👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  ⚽   +    +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    ⚽   +   👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    ⚽  👕  

  +   👕    +    🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    +    +  

  +    +    +   👕  

  +   👕    ⚽   🥅  


  +    +   👕    +  

  +    +    +   👕  

  +   👕    + 

In [135]:
## Highest return = max(return_path1, return_path2)

In [136]:
## Selecting a path is making a sequence of decisions.
## Enumerating all paths is a catastrophic combinatorial explosion.
## The trick is to calculate the action with the highest **return** for every state.

In [137]:
import pandas as pd
REWARDS_TBL = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

In [138]:
REWARDS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,None,None,None,None


In [139]:
for state in REWARDS_TBL.index:
  if not foolsball.__is_terminal_state__(state):
    for action in REWARDS_TBL.columns:
      next_state = foolsball.__get_next_state_on_action__(state,action)
      REWARDS_TBL.loc[state, action] = foolsball.__get_reward_for_transition__(state, next_state)

In [140]:
terminal_states = foolsball.opponents_states+[foolsball.goal_state]
print(terminal_states)
REWARDS_TBL

[2, 7, 9, 15, 17, 19]


,n,e,w,s
0,0,0,0,0
1,0,-5,0,0
2,None,None,None,None
3,0,0,-5,-5
4,0,0,0,0
5,0,0,0,-5
6,-5,-5,0,0
7,None,None,None,None
8,0,-5,0,0
9,None,None,None,None


In [141]:
RETURNS_TBL = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

In [142]:
RETURNS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,None,None,None,None
8,None,None,None,None
9,None,None,None,None


In [143]:
RETURNS_TBL.loc[terminal_states]

,n,e,w,s
2,None,None,None,None
7,None,None,None,None
9,None,None,None,None
15,None,None,None,None
17,None,None,None,None
19,None,None,None,None


In [144]:
RETURNS_TBL.loc[terminal_states] = 0

In [145]:
RETURNS_TBL

,n,e,w,s
0,None,None,None,None
1,None,None,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [146]:
def make_returns_table(terminal_states):
  table = pd.DataFrame.from_dict({s:{a:None for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
  table.loc[terminal_states] = 0
  return table

In [147]:
def fill_returns_table_v0(table,state,debug=False): 
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)

      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      fill_returns_table_v0(table, next_state, debug) # <= Earth shaking problem here!!! 😱😱😱
      table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')
  

In [148]:
table = make_returns_table(terminal_states)
fill_returns_table_v0(table,state=0)

RecursionError: ignored

In [149]:
def fill_returns_table_v1(table,state,debug=False):
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)
      
      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      if next_state == state:
        table.loc[state][action] = -np.inf # <= No self recursion
      else:
        fill_returns_table_v1(table,next_state,debug)
        table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')

In [150]:
table = make_returns_table(terminal_states)
fill_returns_table_v1(table, state=0, debug=False)

RecursionError: ignored

In [151]:
def fill_returns_table_using_v2(table,state, debug=False):
  for action in table.columns:
    if table.loc[state][action] is None:
      next_state = foolsball.__get_next_state_on_action__(state, action)
      reward = foolsball.__get_reward_for_transition__(state, next_state)
      
      if debug:
        print(f'Trying to fill ({state},{action},{next_state})')
      
      if next_state == state:
        table.loc[state][action] = -np.inf # <= No self recursion
      
      elif not table.loc[next_state].isna().any(): # <= No recursion beyond immediate neighbor!
        table.loc[state][action]  = reward + HYPER_PARAMS['gamma'] * table.loc[next_state].max()
    
    else:
      if debug:
        print((state,action),f'already has a RETURN {table.loc[state][action]}')

In [152]:
table = make_returns_table(terminal_states)

In [153]:
fill_returns_table_using_v2(table,state=0)

In [154]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,None,None,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [155]:
fill_returns_table_using_v2(table,state=1)

In [156]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,None,None,None,None
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [157]:
fill_returns_table_using_v2(table,state=3)

In [158]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,None,None
5,None,None,None,None
6,None,None,None,None
7,0,0,0,0
8,None,None,None,None
9,0,0,0,0


In [159]:
for s in range(4,19):
  fill_returns_table_using_v2(table,state=s)

In [160]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [161]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

In [162]:
for s in range(0,19):
  fill_returns_table_using_v2(table,state=s)

In [163]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [164]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

In [165]:
for s in range(0,19):
  fill_returns_table_using_v2(table,state=s)

In [166]:
table

,n,e,w,s
0,-inf,None,-inf,None
1,-inf,-5,None,None
2,0,0,0,0
3,-inf,-inf,-5,-5
4,None,None,-inf,None
5,None,None,None,-5
6,-5,-5,None,None
7,0,0,0,0
8,None,-5,-inf,None
9,0,0,0,0


In [167]:
table.isna().sum()

n    8
e    6
w    6
s    8
dtype: int64

In [169]:
def collect_episode():
  state = foolsball.reset()
  done = False
  episode = []

  while not done:
    action = np.random.choice(foolsball.actions)
    next_state, reward, done = foolsball.step(action)
    episode.append([state, action, reward])
    state = next_state
  
  return episode

In [170]:
ep = collect_episode()
foolsball.render()
print(ep)

  +    +   👕    +  

  +    +    +   👕  

  +    ❗    +    +  

  +    +    +   👕  

  +   👕    +    🥅  


[[0, 'e', 0], [1, 's', 0], [5, 'n', 0], [1, 'w', 0], [0, 's', 0], [4, 'e', 0], [5, 's', -5]]


In [172]:
list(zip(*ep))

[(0, 1, 5, 1, 0, 4, 5),
 ('e', 's', 'n', 'w', 's', 'e', 's'),
 (0, 0, 0, 0, 0, 0, -5)]

In [189]:
def discounted_return_from_episode(ep, gamma=0):
  states, actions, rewards = list(zip(*ep))
  rewards = np.asarray(rewards)
  discount_coeffs = np.asarray([np.power(gamma,p) for p in range(len(rewards))])
  
  l = len(rewards)
  discounted_returns = [np.dot(rewards[i:],discount_coeffs[:l-i]) for i in range(l)]

  return (states, actions, discounted_returns)


In [191]:
discounted_return_from_episode(ep, gamma=HYPER_PARAMS['gamma'])

((0, 1, 5, 1, 0, 4, 5),
 ('e', 's', 'n', 'w', 's', 'e', 's'),
 [-2.6572050000000003,
  -2.9524500000000002,
  -3.2805,
  -3.6450000000000005,
  -4.050000000000001,
  -4.5,
  -5.0])

In [197]:
## Estimate returns by simulating lot of episodes.

# Create empty returns table 
ESTIMATED_RETURNS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')
VISITS_COUNTS_TBL = pd.DataFrame.from_dict({s:{a:0 for a in foolsball.actions} for s in range(foolsball.n_states)}, orient='index')

n_episodes = 100

for i in range(n_episodes):
  episode_i = collect_episode()
  states, actions, discounted_returns = discounted_return_from_episode(episode_i, gamma=HYPER_PARAMS['gamma'])

  for s,a,ret in zip(states, actions, discounted_returns):
    ESTIMATED_RETURNS_TBL.loc[s,a] += ret
    VISITS_COUNTS_TBL.loc[s,a] += 1
  

In [205]:
estimated_returns = ESTIMATED_RETURNS_TBL.div(VISITS_COUNTS_TBL+1) ## Averaging returns. Avoid dividing by zeros.
estimated_returns

,n,e,w,s
0,-5.093780,-4.152160,-4.948062,-3.682993
1,-5.044601,-4.880952,-3.787705,-3.620743
2,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,-3.991854,-3.790532,-4.691764,-3.715818
5,-3.464363,-3.854371,-3.827576,-4.750000
6,-3.333333,-4.545455,-3.231949,-3.429000
7,0.000000,0.000000,0.000000,0.000000
8,-3.131185,-4.687500,-4.757703,-2.997992
9,0.000000,0.000000,0.000000,0.000000


In [200]:
### Policies!!

In [209]:
def greedy_policy_from_returns_tbl(table):
  policy = {s:None for s in table.index }

  for state in table.index:
    if state not in terminal_states:
      greedy_action_index = table.loc[state].argmax()
      greedy_action = table.columns[greedy_action_index]
      policy[state] = greedy_action

  return policy

In [210]:
policy0 = greedy_policy_from_returns_tbl(estimated_returns)

In [211]:
policy0

{0: 's',
 1: 's',
 2: None,
 3: 'n',
 4: 's',
 5: 'n',
 6: 'w',
 7: None,
 8: 's',
 9: None,
 10: 'e',
 11: 'n',
 12: 'n',
 13: 'w',
 14: 's',
 15: None,
 16: 'n',
 17: None,
 18: 'e',
 19: None}

In [212]:
def pretty_print_policy(policy):
  direction_repr = {'n':' 🡑 ', 'e':' 🡒 ', 'w':' 🡐 ', 's':' 🡓 ', None:' ⬤ '}
  out = np.array(shape=[foolsball.n_rows,foolsball.n_cols], dtype='<U3')

  for row in range(out.shape[0]):
    for col in range(out.shape[1]):
      state = row * foolsball.n_cols + col
      out[row,col] = direction_repr[policy[state]]

  print(out)

In [213]:
pretty_print_policy(policy0)

TypeError: ignored

In [215]:
 out = np.array(shape=[foolsball.n_rows,foolsball.n_cols])

5